In [20]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pandas as pd
from sovereign.flood import simple_risk_overlay, flopros_risk_overlay
from sovereign.utils import sum_rasters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set filepaths and provide data info

In [21]:
# Set filepaths
root = Path.cwd().parent # find project root
flood_dir = os.path.join(root, 'inputs', 'flood', 'maps')
adaptation_dir = os.path.join(root, 'inputs', 'flood', 'adapation')
exposure_dir = os.path.join(root, 'outputs', 'exposure')
vulnerability_path = os.path.join(root, 'inputs', 'flood', 'vulnerability', 'jrc_depth_damage.csv')
# Dictionary of flood maps
flood_dic = {10: 'THA_jrc-flood_RP10.tif',
            20: 'THA_jrc-flood_RP20.tif',
            50: 'THA_jrc-flood_RP50.tif',
            75: 'THA_jrc-flood_RP75.tif',
            100: 'THA_jrc-flood_RP100.tif',
            200: 'THA_jrc-flood_RP200.tif',
            500: 'THA_jrc-flood_RP500.tif'}
# Capital stock paths
priv_inf = os.path.join(exposure_dir, 'inf_priv_capstock.tif')
priv_res = os.path.join(exposure_dir, 'res_priv_capstock.tif')
priv_nres = os.path.join(exposure_dir, 'nres_priv_capstock.tif')
pub_inf = os.path.join(exposure_dir, 'inf_pub_capstock.tif')
pub_nres = os.path.join(exposure_dir, 'nres_pub_capstock.tif')
# GVA paths
agr_gva = os.path.join(exposure_dir, 'THA_agr_GVA.tif')
man_gva = os.path.join(exposure_dir, 'THA_man_GVA.tif')
ser_gva = os.path.join(exposure_dir, 'THA_ser_GVA.tif')
# Flood risk map folder
risk_results = os.path.join(root, 'outputs', 'flood', 'risk', 'maps')

In [22]:
# Extract vulnerability curves
vuln_df = pd.read_csv(vulnerability_path)
v_heights = vuln_df['flood_depth'].to_list()
v_res = vuln_df['asia_residential'].to_list()
v_com = vuln_df['asia_commercial'].to_list()
v_ind = vuln_df['asia_industrial'].to_list()
v_agr = vuln_df['asia_agriculture'].to_list()
v_inf = vuln_df['asia_infrastructure'].to_list()

#### Run risk for baseline protection scenario

In [23]:
# Check to see if results directory exists
if not os.path.exists(baseline_results):
    os.makedirs(baseline_results)
# Loop over return periods and perform risk calculation
for RP in flood_dic:
    priv_inf_risk = os.path.join(risk_results, f'JRC_{RP}_priv_inf_cap_damages.tif')
    priv_res_risk = os.path.join(risk_results, f'JRC_{RP}_priv_res_cap_damages.tif')
    priv_nres_risk = os.path.join(risk_results, f'JRC_{RP}_priv_nres_cap_damages.tif')
    pub_inf_risk = os.path.join(risk_results, f'JRC_{RP}_pub_inf_cap_damages.tif')
    pub_nres_risk = os.path.join(risk_results, f'JRC_{RP}_pub_nres_cap_damages.tif')
    agr_gva_risk = os.path.join(risk_results, f'JRC_{RP}_agr_gva_losses.tif')
    man_gva_risk = os.path.join(risk_results, f'JRC_{RP}_man_gva_losses.tif')
    ser_gva_risk = os.path.join(risk_results, f'JRC_{RP}_ser_gva_losses.tif')
    flood_path = os.path.join(flood_dir, flood_dic[RP])
    simple_risk_overlay(flood_path, priv_inf, priv_inf_risk, [v_heights, v_inf])
    simple_risk_overlay(flood_path, priv_res, priv_res_risk, [v_heights, v_res])
    simple_risk_overlay(flood_path, priv_nres, priv_nres_risk, [v_heights, v_com])
    simple_risk_overlay(flood_path, pub_inf, pub_inf_risk, [v_heights, v_inf])
    simple_risk_overlay(flood_path, pub_nres, pub_nres_risk, [v_heights, v_com])
    simple_risk_overlay(flood_path, agr_gva, agr_gva_risk, [v_heights, v_agr])
    simple_risk_overlay(flood_path, man_gva, man_gva_risk, [v_heights, v_ind])
    simple_risk_overlay(flood_path, ser_gva, ser_gva_risk, [v_heights, v_com])
    # We are the going to sum public and private capital stock into individual risk maps 
    sum_rasters([priv_inf_risk, priv_res_risk, priv_nres_risk], os.path.join(risk_results, f'JRC_{RP}_priv_cap_damages.tif'))
    sum_rasters([pub_inf_risk, pub_nres_risk], os.path.join(risk_results, f'JRC_{RP}_pub_cap_damages.tif'))